In [1]:
###### import pandas, numpy, json, pprint, and requests #############
import pandas as pd
import numpy as np
import json
import pprint
import requests
from bs4 import BeautifulSoup

pd.set_option('display.width', 80)
pd.set_option('display.max_columns',6)

In [2]:
####### parse the web page and get the header row of the table ######################

webpage = requests.get("http://www.alrb.org/datacleaning/covidcaseoutliers.html")
bs = BeautifulSoup(webpage.text, 'html.parser')
theadrows = bs.find('table', {'id':'tblDeaths'}).thead.find_all('th') # we get rows and columns over here
type(theadrows)
labelcols = [j.get_text() for j in theadrows]
labelcols[0] = "rowheadings"
labelcols

# //find method to get the table we want
# //soup = BeautifulSoup(data, 'html.parser') // soup uses html parser to parse the html doc
# //Beautiful Soup's find method to get the table we want.
# //find_all to retrieve the elements nested within the th elements for that table.
# //th is the variable here we can put anything.

#//  can get one HTML element with find and get one or more with find_all.

['rowheadings',
 'Cases',
 'Deaths',
 'Cases per Million',
 'Deaths per Million',
 'population',
 'population_density',
 'median_age',
 'gdp_per_capita',
 'hospital_beds_per_100k']

In [3]:
theadrows

[<th align="center" height="18" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="top"><b><font color="#000000">Country</font></b></th>,
 <th align="center" sdnum="1033;0;0" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="top"><b><font color="#000000">Cases</font></b></th>,
 <th align="center" sdnum="1033;0;0" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="top"><b><font color="#000000">Deaths</font></b></th>,
 <th align="center" sdnum="1033;0;0" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="top"><b><font color="#000000">Cases per Million</font></b></th>,
 <th align="center" sdnum="1033;0;0" style="border

In [4]:
########## get the data from the table cells####################
rows = bs.find('table', {'id':'tblDeaths'}).tbody.find_all('tr')
datarows = []
labelrows = []
for row in rows:
  rowlabels = row.find('th').get_text()
  cells = row.find_all('td', {'class':'data'})
  if (len(rowlabels)>3):
    labelrows.append(rowlabels)
  if (len(cells)>0):
    cellvalues = [j.get_text() for j in cells]
    datarows.append(cellvalues)

pprint.pprint(datarows[0:2])
pprint.pprint(labelrows[0:2])

for i in range(len(datarows)):
  datarows[i].insert(0, labelrows[i])

pprint.pprint(datarows[0:2])

[['9,394', '653', '214', '15', '43,851,043', '17', '29', '13,914', '1.9'],
 ['16,642', '668', '1848', '74', '9,006,400', '107', '44', '45,437', '7.4']]
['Algeria', 'Austria']
[['Algeria',
  '9,394',
  '653',
  '214',
  '15',
  '43,851,043',
  '17',
  '29',
  '13,914',
  '1.9'],
 ['Austria',
  '16,642',
  '668',
  '1848',
  '74',
  '9,006,400',
  '107',
  '44',
  '45,437',
  '7.4']]


In [5]:
#// finding all of the table rows for the table we want
# // for each row find th element and retrieve the text
#// use the above text for row labels
#// th= table header, tr= table row, td= table data.


In [6]:
rows

[<tr>
 <th align="left" height="18" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="bottom"><font color="#000000">Algeria</font></th>
 <td align="right" class="data" sdnum="1033;0;0" sdval="9394" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="bottom"><font color="#000000">9,394</font></td>
 <td align="right" class="data" sdnum="1033;0;0" sdval="653" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="bottom"><font color="#000000">653</font></td>
 <td align="right" class="data" sdnum="1033;0;0" sdval="214.225" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: 1px solid #000000" valign="bottom"><font color="#000000">214</font></td>
 <td a

In [7]:
rowlabels

'United States'

In [10]:
######### load the data into pandas######################
totaldeaths = pd.DataFrame(datarows, columns=labelcols) # data rows list to data frame
totaldeaths.iloc[:,1:5].head()
totaldeaths.dtypes
totaldeaths.columns = totaldeaths.columns.str.replace(" ", "_").str.lower()

for col in totaldeaths.columns[1:-1]:
  totaldeaths[col] = totaldeaths[col].\
    str.replace("[^0-9]","").astype('int64') #Fix the column names and convert the data to numeric values.

totaldeaths['hospital_beds_per_100k'] = totaldeaths['hospital_beds_per_100k'].astype('float')
totaldeaths.head()
totaldeaths.dtypes

#// Remove spaces from column names. Remove all non-numeric data from the
#// first columns with data, including the commas (str.replace("[^0-9]",""). Convert to numeric values, except for the rowheadings column

<ipython-input-10-10c95f5f81fd>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  str.replace("[^0-9]","").astype('int64') #Fix the column names and convert the data to numeric values.


rowheadings                object
cases                       int64
deaths                      int64
cases_per_million           int64
deaths_per_million          int64
population                  int64
population_density          int64
median_age                  int64
gdp_per_capita              int64
hospital_beds_per_100k    float64
dtype: object